In [ ]:
import torch
import importlib
import callback, my_attn, my_pipeline
importlib.reload(callback)
importlib.reload(my_attn)
importlib.reload(my_pipeline)
from my_pipeline import run_with_attn_replacement, MutualAttention
from diffusers import StableDiffusionPipeline, DDIMInverseScheduler, DDIMScheduler
from my_attn import prep_unet_attention
from tqdm.notebook import tqdm
from callback import QKVRecordCallback
from utils import load_image, img_to_latents, concat_img 
from callback import ATTN_BLOCKS
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda'
dtype = torch.float16
model_dir = '/home/tom/fshare/models/Stability-AI/stable-diffusion-2-1'
# model_dir = '/home/tom/fshare/models/runwayml/stable-diffusion-v1-5'

pipe = StableDiffusionPipeline.from_pretrained(
    model_dir, safety_checker=None, torch_dtype=dtype)

pipe = pipe.to(device)

In [ ]:
num_inversion_steps = 999
prep_unet_attention(pipe.unet)
imgname = 'images/kitten.jpg'
input_img = load_image(
    imgname,
    target_size=(512, 512)
).to(device=device, dtype=dtype)
prompt_inversion = ''

with torch.no_grad():
    pipe.scheduler = DDIMInverseScheduler.from_pretrained(
        '/home/tom/fshare/models/Stability-AI/stable-diffusion-2-1', subfolder='scheduler')
    latents = img_to_latents(input_img, pipe.vae)  # VAE latent
    inv_latents, _ = pipe(
        prompt=prompt_inversion,
        guidance_scale=1,
        width=input_img.shape[-1],
        height=input_img.shape[-2],
        output_type='latent',
        return_dict=False,
        num_inference_steps=num_inversion_steps,
        latents=latents,
    )

In [ ]:
mutual_attn = MutualAttention(pipe)
result = mutual_attn(
    replaced_attn_indice=[12],
    replaced_qkv=['k', 'v'],
    end=500,
    prompt='A kitten is sitting in a basket on a branch',
    ref_prompt='A kitten is sitting in a basket on a branch',
    latents=inv_latents,
    ref_latents=inv_latents,
)